In [20]:
# Import Modules
import pandas as pd
import numpy as np

import datetime
import dateparser
from datetime import date, timedelta

# Import Modules
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import sys
# adding Notebooksfolder to the system path
sys.path.insert(0, '/Users/emil/miniforge3/envs/googleapi/Notebooks')

import importlib
import dates_funcs
importlib.reload(dates_funcs)
from dates_funcs import appendDFToCSV


In [21]:
# Function to fully display dataframe
def force_show_all(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
        display(HTML(df.to_html()))
# ... now when you're ready to fully display df:
#force_show_all(df)

In [22]:
#FRAN https://www.shortautomaton.com/connecting-to-google-search-console-api-with-python/ 

# Define function to get authorization
def gsc_auth(scopes):
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', scopes)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', scopes)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('searchconsole', 'v1', credentials=creds)

    return service

scopes = ['https://www.googleapis.com/auth/webmasters.readonly']

service = gsc_auth(scopes)

Query Function

In [27]:
def query_full(service, site_url, payload):
    """Run a query on the Google Search Console API and return a dataframe of results.
    
    Args:
        service (object): Service object from connect()
        site_url (string): URL of Google Search Console property
        payload (dict): API query payload dictionary
    
    Return:
        df (dataframe): Pandas dataframe containing requested data. 
    
    """
    
    response = service.searchanalytics().query(siteUrl=site_url, body=payload).execute()
    
    results = []
    
    for row in response['rows']:    
        data = {}
        
        for i in range(len(payload['dimensions'])):
            data[payload['dimensions'][i]] = row['keys'][i]

        data['clicks'] = row['clicks']
        data['impressions'] = row['impressions']
        data['ctr'] = round(row['ctr'] * 100, 2)
        data['position'] = round(row['position'], 2)        
        results.append(data)
    
    return pd.DataFrame.from_dict(results)

Find latest Date in old CSV

In [24]:
# Find maximum date in csv file.

# Read file in chunks (parts)
df = pd.read_csv("output/csv/testa.csv", chunksize=100000)

# Max date to start comparing to
#max_date_overall = testdateUS.strftime("%Y-%m-%d")
max_date_overall = datetime.datetime(2022, 1, 1).date()

#time.strptime(date1, "%d/%m/%Y")

print(max_date_overall)


for chunk in df:
    chunk['date'] = chunk['date']
    max_date_in_chunk = max(chunk['date'])
    
    max_date_in_chunk = datetime.datetime.strptime(max_date_in_chunk,'%Y-%m-%d').date()
    
    if max_date_in_chunk > max_date_overall:
        max_date_overall = max_date_in_chunk
    
    print('maxdate in chunk: ',max_date_overall) 

test_start_date_name = '2022-01-01'
test_start_date = dateparser.parse(test_start_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

test_end_date_name = 'three days ago'
test_end_date = dateparser.parse(test_end_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

print('maxdate in csv: ',max_date_overall) 
print('start date given: ',test_start_date)
print('end date given: ',test_end_date)

start_fetch_date = max_date_overall+datetime.timedelta(days=1)
start_fetch_date = start_fetch_date.strftime("%Y-%m-%d")

if test_start_date > max_date_overall:
    print('start date is after largest date in csv,', end='\n')
    print('will fetch updated data from date: ', start_fetch_date, end='\n')   
    print('to : ', test_end_date)
else:
    print('data already fetched, wait a day before running again')

print(max_date_overall+datetime.timedelta(days=1))

print(start_fetch_date)
print(test_end_date)

FileNotFoundError: [Errno 2] No such file or directory: 'output/csv/testa.csv'

In [36]:
# Now to print to log when program running 
nowDT = datetime.datetime.now()
now = datetime.datetime.strftime(nowDT,'%Y-%m-%d_%H:%M:%S')
print("Script started: "+now)

# Check of Dates
f = open("latestDate.txt")
maxSavedDate = f.read()
f.close()
#print("Previously fetched data up to and including: "+ maxSavedDate)
maxSavedDateDT = datetime.datetime.strptime(maxSavedDate,'%Y-%m-%d').date()
print("Last date saved: "+maxSavedDate)

# Todays date
todayDT = date.today()
today = datetime.datetime.strftime(todayDT,'%Y-%m-%d')

three_days_agoDT = todayDT + datetime.timedelta(days=-3)
three_days_ago = datetime.datetime.strftime(three_days_agoDT,'%Y-%m-%d')

# start_date as the next day as maxSavedDate
test_start_dateDT = maxSavedDateDT + datetime.timedelta(days=1)
test_start_date = datetime.datetime.strftime(test_start_dateDT,'%Y-%m-%d')
print("Fetching new data, starting: "+test_start_date)

test_end_dateDT = todayDT + datetime.timedelta(days=-10)
test_end_date = datetime.datetime.strftime(test_end_dateDT,'%Y-%m-%d')
# one days after start_date, to get data for two days
# end_date as the day after start_date

print("up to and including: "+test_end_date)

Script started: 2023-06-12_21:21:04
Last date saved: 2022-01-27
Fetching new data, starting: 2022-01-28
up to and including: 2023-06-02


In [147]:
# First Dates with data available

#max_date_overall_name = '2022-01-27'
#max_date_overall = dateparser.parse(max_date_overall_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

#test_start_date_name = '2022-01-28'
#test_start_date = dateparser.parse(test_start_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

#test_end_date_name = 'ten days ago'
#test_end_date = dateparser.parse(test_end_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

LOOPA TEST!

In [42]:
# Loop to send query between start_date and end_date-1 day. Each date to a new DF in dfs. 
# Then concatenated into final DF.

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Only run if dates are not already in old csv.        
        
if start_dateDT > maxSavedDateDT and three_days_agoDT > end_dateDT and three_days_agoDT > maxSavedDateDT:
        
    start_date = start_dateDT 
    end_date = test_end_dateDT

    dfs = []

    for query_date in daterange(start_date, end_date):
        query_date = query_date.strftime("%Y-%m-%d")

        payload = {
            'startDate': query_date,
            'endDate': query_date,
            'dimensions': ["date","page","device","query","country"],
            'rowLimit': 25000,
            'startRow': 0
            }

        site_url = "https://advokatfamiljforsvar.se"

        try:
        
            df = query_full(service, site_url, payload)
            dfs.append(df)

        except KeyError:

            break

        # New DF med all data fran start_date - end_data-1 
        df_new = pd.concat(dfs, ignore_index=True)
        df_new.reset_index(inplace=True, drop=True)
            
        # Sort columns in same order as the file to write to
        df_new.sort_values(by=["date","page","device","query","country"],ascending=True,inplace=True)
            
        # Columns to case matching reference and analytics data
        df_new['device'] = df_new['device'].str.lower()
        df_new['country'] = df_new['country'].str.upper()
            
        # Column names to matching reference and analytics data
        df_new.rename(columns = {'date':'date','country':'countryIdLong', 'page':'landingPage', 'device':'deviceCategory'}, inplace = True)
            
        # To File
        csvFilePath = "output/csv/"
        fileName = "testa.csv"
        sep = ","
            
        dates_funcs.appendDFToCSV(df, csvFilePath, fileName, sep)
     
else:
    
    print('Wait a day, before running again')
    #display(df_new.tail())

In [29]:
# Deep copy of df
df_test = df_new.copy()

NameError: name 'df_new' is not defined

In [159]:

# Sort out Date
df_sorted = dates_funcs.sortOut_date_searchFormat(df_test)

In [164]:
# Split by years
df_by_years = dates_funcs.split_years(df_sorted)

In [166]:
df2022 = df_by_years[0].copy()

In [167]:
df2023 = df_by_years[1].copy()

In [172]:
df_2022_by_month = dates_funcs.split_months(df2022)

In [173]:
df_2023_by_month = dates_funcs.split_months(df2023)

In [182]:
df_2023_by_month[5]

,dateFull,year,quarter,month,monthName,week,weekday,weekdayName,landingPage,deviceCategory,query,countryIdLong,clicks,impressions,ctr,position
734376,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,IND,0,2,0.0,55.5
734377,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,JPN,0,1,0.0,94.0
734378,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,KOR,0,1,0.0,79.0
734379,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,PRI,0,1,0.0,91.0
734380,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,SWE,0,1,0.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753918,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,desktop,verkställighet,USA,0,2,0.0,25.5
753919,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,umgängessabotage,NZL,0,1,0.0,36.0
753920,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,vad är verkställighet,SWE,0,1,0.0,12.0
753921,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,verkställighet,DEU,0,1,0.0,26.0


In [ ]:
 ## Max time to latestDatefile
    
# Find largest date in dataframes, looking at df eventName, which should have all dates. (?)
maxTimestamp = dfs_merged_fillNA[14]["dateFull"].max()
maxTimestampString = datetime.datetime.strftime(maxTimestamp,'%Y-%m-%d')
maxDate = datetime.datetime.strptime(maxTimestampString,'%Y-%m-%d').date()
    
print("Fetched data until "+maxTimestampString)
    
    # If date is larger than priviously max date.
if maxDate > maxSavedDateDT:
    # Write largest date to file
     f = open("latestDate.txt", 'w')
      f.write(maxTimestampString)
      f.close()
    
print("Written to .csv files")
print("\n")

In [33]:
for query_date in daterange(start_date, end_date):
        query_date = query_date.strftime("%Y-%m-%d")
        print(query_date)

2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-19
2022-02-20
2022-02-21
2022-02-22
2022-02-23
2022-02-24
2022-02-25
2022-02-26
2022-02-27
2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-05
2022-03-06
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-12
2022-03-13
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-19
2022-03-20
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-26
2022-03-27
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-02
2022-04-03
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-09
2022-04-10
2022-04-11
2022-04-12
2022-04-13
2022-04-14
2022-04-15
2022-04-16
2022-04-17
2022-04-18
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-23
2022-04-24
2022-04-25
2022-04-26
2022-04-27
2022-04-28